## Example used in the blog (Physics & Computer, Love.) article about Matrix Factorization.
Read more about: https://physicscomputerlove.com/en/machine-learning/matrix-factorization/

In [1]:
import examples as ex
from examples import pd, np
from IPython.display import display

## 1. Load sample data of user interactions

In [2]:
matrix_interactions = ex.get_matrix_interaction()
matrix_interactions

rating                                  \
music       Another One Bites the Dust Back in Black Bohemian Rhapsody   
user                                                                     
Darth Vader                        4.0           4.0               2.0   
Hermione                           1.0           0.0               5.0   
Ripley                             5.0           5.0               2.0   
Spock                              2.0           0.0               5.0   

                                                                       \
music       Brandenburg Concerto No. 3 Clair de Lune Comfortably Numb   
user                                                                    
Darth Vader                        0.0           1.0              2.0   
Hermione                           0.0           5.0              2.0   
Ripley                             1.0           2.0              3.0   
Spock                              5.0           5.0              4.0   

                                                                             \
music       Don't Stop Believin' Firework Hedwig’s Theme O Fortuna Survivor   
user                                                                          
Darth Vader                  2.0      0.0            0.0       4.0      0.0   
Hermione                     2.0      5.0            5.0       2.0      2.0   
Ripley                       4.0      4.0            0.0       5.0      5.0   
Spock                        3.0      2.0            3.0       4.0      2.0   

                                
music       The Imperial March  
user                            
Darth Vader                5.0  
Hermione                   1.0  
Ripley                     4.0  
Spock                      0.0

## 2. Initialize the user matrix and item matrix

In [3]:
ratings = matrix_interactions.values
k = 5 # numer of latent dimensions

num_users, num_items = ratings.shape

# Initialize user and item latent feature matrices
user_matrix = np.random.normal(scale=1. / k, size=(num_users, k))  # U matrix
item_matrix = np.random.normal(scale=1. / k, size=(num_items, k))  # V matrix

# Display the random latent features for users and items
display(pd.DataFrame(user_matrix).style.set_caption("Random latent features for users"))
display(pd.DataFrame(item_matrix).style.set_caption("Random latent features for items"))


,0,1,2,3,4
0,0.039838,-0.007888,0.170533,-0.368997,-0.122628
1,-0.093917,0.166281,-0.336085,0.286904,-0.117067
2,-0.046059,-0.117961,-0.189409,-0.186814,-0.008404
3,-0.252922,0.326049,-0.340885,0.182133,0.004918


,0,1,2,3,4
0,0.453741,0.256213,-0.152726,0.131163,0.093160
1,-0.013445,0.126091,-0.198593,-0.016365,0.007979
2,0.165660,0.110555,0.108378,-0.019994,0.145744
3,-0.135834,0.041819,0.213143,-0.075233,-0.018873
4,-0.484212,0.123287,-0.234250,0.358887,0.190027
5,-0.318987,-0.072454,0.048965,0.230088,0.150034
6,-0.113358,0.013525,0.183116,-0.144595,-0.257412
7,0.166350,-0.169958,-0.033311,-0.050493,0.167346
8,0.185286,-0.040875,-0.045004,0.175524,0.449542
9,-0.158091,0.127592,-0.016239,-0.428960,0.376738


## 3. Error Maximization and Optimization Methods


### 3.1. Implementation of the SGD algorithm

In [4]:
alpha = 0.01 # learning rate
lambda_ = 0.02 # regularization term

"""
- Returns the predicted rating of user i for item j.
- It does this through the dot product between the user feature vector (users[i, :]) and the item feature vector (items[j, :]).
- The resulting vector indicates the affinity between the user and the item.
"""
def predict(users, items, i, j):
    return np.dot(users[i, :], items[j, :].T)

"""
- This function performs training using Stochastic Gradient Descent (SGD).
- ratings.nonzero() returns the indices (i, j) of the non-zero positions in the ratings array, i.e. where there is a known rating.
- ratings[i, j]: actual value of the rating given by user i to item j.
- predict(users, items, i, j): rating prediction calculated by the model.
- eij: diferença entre o valor real e o previsto.
- The term 2 * eij * items[j, :] corrects the direction of the user vector based on the prediction error.
- The term - lambda_ * users[i, :] is the regularization to avoid overfitting.
- After the updates, the function returns the adjusted users and items vectors, which contain the final user and item embeddings.
"""
def sgd(ratings, users, items):
    for i, j in zip(*ratings.nonzero()):
        eij = ratings[i,j] - predict(users, items, i, j)
        users[i, :] += alpha * (2 * eij * items[j, :] - lambda_ * users[i, :])
        items[j, :] += alpha * (2 * eij * users[i, :] - lambda_ * items[j, :])

    return users, items

sgd(ratings, user_matrix, item_matrix)

(array([[ 0.03206509,  0.02504244,  0.16546316, -0.37740867, -0.0716378 ],
        [-0.10346349,  0.17218407, -0.34004188,  0.31376894, -0.00725238],
        [-0.05443973, -0.07920718, -0.19657119, -0.20597252,  0.05848069],
        [-0.30665292,  0.36221949, -0.34260886,  0.19311063,  0.09668474]]),
 array([[ 0.44883756,  0.26417774, -0.18027904,  0.09683633,  0.08187418],
        [-0.00740279,  0.12006147, -0.20919163, -0.0632145 , -0.00125879],
        [ 0.14017156,  0.16201542,  0.03795361,  0.00581674,  0.13344453],
        [-0.159131  ,  0.07584143,  0.17643908, -0.06120126, -0.01681229],
        [-0.51978977,  0.17401647, -0.30540631,  0.39356114,  0.18325133],
        [-0.34707523, -0.04007506,  0.00132005,  0.23665033,  0.14772956],
        [-0.13941537,  0.0391098 ,  0.1374393 , -0.14420906, -0.26311789],
        [ 0.13934539, -0.14386395, -0.09901736, -0.02265374,  0.1625374 ],
        [ 0.15817379, -0.00285801, -0.09982418,  0.22229342,  0.4505526 ],
        [-0.18840026,  

### 3.2. MSE (Mean Squared Error)


In [5]:
"""
- This variable will accumulate the squared error of each pair (i, j) in the ratings matrix.
- ratings.nonzero() returns the indices (i, j) where the ratings are known (nonzero).
- ratings[i, j]: actual rating given by user i to item j.
- (ratings[i, j] - y_pred) ** 2: squared error between actual and predicted value.
"""
def compute_loss(ratings, users, items):
    error = 0
    for i, j in zip(*ratings.nonzero()):
        y_pred = predict(users, items, i, j)
        error += (ratings[i, j] - y_pred) ** 2

    mse = np.mean(error)

    return mse

compute_loss(ratings, user_matrix, item_matrix)

np.float64(495.6434401126616)

### 3.3. Regularization

In [6]:
"""
- This variable will accumulate the squared error of each pair (i, j) in the ratings matrix.
- ratings.nonzero() returns the indices (i, j) where the ratings are known (nonzero).
- ratings[i, j]: actual rating given by user i to item j.
- (ratings[i, j] - y_pred) ** 2: squared error between actual and predicted value.
"""
def compute_loss(ratings, users, items):
    error = 0
    for i, j in zip(*ratings.nonzero()):
        y_pred = predict(users, items, i, j)
        error += (ratings[i, j] - y_pred) ** 2

    mse = np.mean(error)

    # Regularization term to avoid overfitting
    # Penalizes large magnitudes in user and item vectors
    regularization = lambda_ * (np.linalg.norm(users)**2 + np.linalg.norm(items)**2)

    return mse + regularization

compute_loss(ratings, user_matrix, item_matrix)

np.float64(495.7070118903119)

## 3.4 Test the implementation

In [7]:
epochs = 1000
for epoch in range(epochs):
    user_matrix, item_matrix = sgd(ratings, user_matrix, item_matrix)

    if epoch % 100 == 0:
        error = compute_loss(ratings, user_matrix, item_matrix)
        print(f'Epoch {epoch + 1}/{epochs} - Error: {error:.4f}')

Epoch 1/1000 - Error: 482.7788
Epoch 101/1000 - Error: 1.4654
Epoch 201/1000 - Error: 1.4692
Epoch 301/1000 - Error: 1.4916
Epoch 401/1000 - Error: 1.5133
Epoch 501/1000 - Error: 1.5332
Epoch 601/1000 - Error: 1.5507
Epoch 701/1000 - Error: 1.5658
Epoch 801/1000 - Error: 1.5787
Epoch 901/1000 - Error: 1.5895


## 4. Recommendations new music

In [8]:
from matrix_factorization import MatrixFactorization
from sklearn.preprocessing import MinMaxScaler

ratings = matrix_interactions.values

scaler = MinMaxScaler(feature_range=(0, 5))
normalized_ratings = scaler.fit_transform(ratings)

model = MatrixFactorization(n_factors=75, learning_rate=0.001, regularization=0.02)
matrix_factorization_result = model.fit(ratings, epochs=600)

Epoch 1/600 - Training Loss: 6.0019
Epoch 2/600 - Training Loss: 6.0011
Epoch 3/600 - Training Loss: 6.0003
Epoch 4/600 - Training Loss: 5.9994
Epoch 5/600 - Training Loss: 5.9986
Epoch 6/600 - Training Loss: 5.9977
Epoch 7/600 - Training Loss: 5.9968
Epoch 8/600 - Training Loss: 5.9959
Epoch 9/600 - Training Loss: 5.9949
Epoch 10/600 - Training Loss: 5.9939
Epoch 11/600 - Training Loss: 5.9928
Epoch 12/600 - Training Loss: 5.9917
Epoch 13/600 - Training Loss: 5.9905
Epoch 14/600 - Training Loss: 5.9893
Epoch 15/600 - Training Loss: 5.9879
Epoch 16/600 - Training Loss: 5.9865
Epoch 17/600 - Training Loss: 5.9851
Epoch 18/600 - Training Loss: 5.9835
Epoch 19/600 - Training Loss: 5.9818
Epoch 20/600 - Training Loss: 5.9800
Epoch 21/600 - Training Loss: 5.9780
Epoch 22/600 - Training Loss: 5.9760
Epoch 23/600 - Training Loss: 5.9738
Epoch 24/600 - Training Loss: 5.9714
Epoch 25/600 - Training Loss: 5.9689
Epoch 26/600 - Training Loss: 5.9661
Epoch 27/600 - Training Loss: 5.9632
Epoch 28/6

In [9]:
df_matrix_interactions = pd.DataFrame(matrix_factorization_result,
                                     index=matrix_interactions.index,
                                     columns=matrix_interactions.columns)

print('Matrix Factorization Result:')
display(df_matrix_interactions)

Matrix Factorization Result:


rating                                  \
music       Another One Bites the Dust Back in Black Bohemian Rhapsody   
user                                                                     
Darth Vader                   4.093801      3.991699          1.683605   
Hermione                      1.023098      3.145756          4.934192   
Ripley                        4.877507      4.974933          2.260267   
Spock                         1.998512      3.763525          4.988831   

                                                                       \
music       Brandenburg Concerto No. 3 Clair de Lune Comfortably Numb   
user                                                                    
Darth Vader                   0.632839      1.062763         2.210652   
Hermione                      4.100897      5.026325         2.118969   
Ripley                        1.031988      1.958296         2.831570   
Spock                         4.968491      4.925842         3.869386   

                                                                               \
music       Don't Stop Believin'  Firework Hedwig’s Theme O Fortuna  Survivor   
user                                                                            
Darth Vader             2.486155  2.502766       1.884188  4.086522  3.786563   
Hermione                2.128347  4.854885       4.897921  2.071073  1.975606   
Ripley                  3.583941  3.954575       2.969706  4.904599  4.951301   
Spock                   2.903590  2.141790       3.070534  3.923460  2.040637   

                                
music       The Imperial March  
user                            
Darth Vader           4.418384  
Hermione              0.909198  
Ripley                4.466129  
Spock                 2.446687

In [10]:
print('Matrix Interactions:')
display(matrix_interactions)

Matrix Interactions:


rating                                  \
music       Another One Bites the Dust Back in Black Bohemian Rhapsody   
user                                                                     
Darth Vader                        4.0           4.0               2.0   
Hermione                           1.0           0.0               5.0   
Ripley                             5.0           5.0               2.0   
Spock                              2.0           0.0               5.0   

                                                                       \
music       Brandenburg Concerto No. 3 Clair de Lune Comfortably Numb   
user                                                                    
Darth Vader                        0.0           1.0              2.0   
Hermione                           0.0           5.0              2.0   
Ripley                             1.0           2.0              3.0   
Spock                              5.0           5.0              4.0   

                                                                             \
music       Don't Stop Believin' Firework Hedwig’s Theme O Fortuna Survivor   
user                                                                          
Darth Vader                  2.0      0.0            0.0       4.0      0.0   
Hermione                     2.0      5.0            5.0       2.0      2.0   
Ripley                       4.0      4.0            0.0       5.0      5.0   
Spock                        3.0      2.0            3.0       4.0      2.0   

                                
music       The Imperial March  
user                            
Darth Vader                5.0  
Hermione                   1.0  
Ripley                     4.0  
Spock                      0.0

## 5. Similarity between users

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
user_similarity = cosine_similarity(matrix_factorization_result)
df_user_similarity = pd.DataFrame(user_similarity, index=ex.users, columns=ex.users)
display(df_user_similarity)

,Ripley,Darth Vader,Spock,Hermione
Ripley,1.000000,0.676900,0.991161,0.782574
Darth Vader,0.676900,1.000000,0.746803,0.926022
Spock,0.991161,0.746803,1.000000,0.818318
Hermione,0.782574,0.926022,0.818318,1.000000


## 6. Recommend new musics to Hermione

In [12]:
def recommend_by_user(user, df_predicted, df_interactions, top_k=3):
    # Calculate user similarity matrix
    user_similarity = cosine_similarity(df_predicted.values)
    user_similarity_df = pd.DataFrame(user_similarity, index=ex.users, columns=ex.users)

    # Get the top similar users
    similarities = user_similarity_df.loc[user].sort_values(ascending=False)[1:top_k + 1]
    print('Sser similarity score:')
    print(similarities)

    # Initialize recommendations
    recommendations = pd.Series(0, index=df_predicted.columns, dtype=np.float64)

    # Aggregate recommendations based on similar users
    for user_similar, similarity in similarities.items():
        recommendations = recommendations.add(
            df_predicted.loc[user_similar].reindex(recommendations.index, fill_value=0) * similarity,
            fill_value=0
        )

    # Filter out already known items
    already_known = df_interactions.loc[user] > 0
    recommendations = recommendations[~already_known.reindex(recommendations.index, fill_value=False)]

    return recommendations.sort_values(ascending=False)

recommend_by_user('Hermione', df_matrix_interactions, matrix_interactions)

Sser similarity score:
Darth Vader    0.926022
Spock          0.818318
Ripley         0.782574
Name: Hermione, dtype: float64


        music                     
rating  Back in Black                 10.669412
        Brandenburg Concerto No. 3     5.459433
dtype: float64